#### Accept - Reject Algorithm

In [ ]:
def acceptreject(p,q, sample_q, n_iter):
    """Samples from a target distribution p using the proposal distribution q via a accept-reject scheme"""
    samples = np.array([])
    for n in range(n_iter):
        x_new = sample_q()    # Generate a new sample from q(x)
        r = p(x_new) / q(x_new)    # Acceptance ratio
        if r > 1:
            # Stop algorithm if acceptance ratio exceeds one
            print("ERROR: Acceptance ratio is greater than one at x = " + str(x_new))
            break;
        if uniform.rvs(0, 1) < r:
            samples = np.append(samples,x_new)
    return samples

#### Importance Sampling

In [ ]:
def importance_sampler(f, p, q, num_samples):
     """Importance sampling routine that estimates the integral of f(x)p(x) using N samples from q(x)"""
    xi = samples(num_samples)
    fi = f(xi)
    wi = p(xi)/q(xi)    # Importance weights
    mu = np.sum(fi*wi)/N    # Estimate of integral
    var_mu = np.var(fi*wi)/N    # Variance of estimate
    stderr = np.sqrt(var_mu)    # Standard error
    return {"mu": mu, "stderr": stderr}

#### Metropolis Hastings

In [ ]:
def metropolis_hastings(p, q, sample_q, param_init, num_samples):
    """Metropolis-Hastings algorithm that generates samples from a target distribution p
    and using a proposal kernel q"""
    samples = np.zeros((num_samples, len(param_init)))
    samples[0] = param_init

    for n in range(num_samples-1):

        x_current = samples[n]
        x_proposed = sample_q(x_current)

        # Metropolis (acceptance) ratio
        r_num = q(x_proposed, x_current) * p(x_proposed)
        r_denom = q(x_current, x_proposed) * p(x_current)
        r = r_num / r_denom

        # Decide whether to accept the proposed move with probability min(1,r)
        if r >= 1:
            x_next = x_proposed
        elif uniform.rvs(0, 1) < r:
            x_next = x_proposed
        else:
            x_next = x_current

        samples[n+1] = x_next    # Store the new sample

    return samples

#### Gibbs Sampler (in 2D)

In [ ]:
def gibbs_sampler_2D(sample_x, sample_y, param_init, num_samples):
    """Gibbs sampler that generates samples from a 2D distribution using the
    conditional distributions P(x|y) = sample_x and P(y|x) = sample_y"""
    samples = np.zeros((num_samples, len(param_init)))
    samples[0] = param_init

    for n in range(num_samples-1):
        samples[n+1][0] = sample_x(samples[n][1])    # Update x
        samples[n+1][1] = sample_y(samples[n+1][0])    # Update y

    return samples

#### Miscellaneous

In [ ]:
def num_bins(x):
    """Selects number of histogram bins selected according to the Freedman–Diaconis rule"""
    N = len(x)
    bin_width = 2*iqr(x)/(N)**(1/3)
    n_bins = int((np.max(x) - np.min(x))/bin_width)
    return n_bins

In [ ]:
def MCMC_stats(x):
    """Returns E[x] along with MCSE and ESS estimates"""
    mean = np.mean(x)    # E[x]
    rho = acf(x, nlags = 100)    # Autocorrelation of the chain from t = 0 to t = 100
    rho = rho[:np.argmax(rho<0)]    # Truncate at first negative
    N_eff = len(x)/(2*np.sum(rho)-1)    # Effect sample size
    mcse = np.std(x)/np.sqrt(N_eff)    # Monte Carlo Standard Error
    print("ESS: ", N_eff)
    print("Mean: ", mean)
    print("MCSE: ", mcse)